<hr>
# Data Cleaning

Notebook for testing the cleaning process for our dataset.
<hr>
## Setup

In [1]:
import pandas as pd
import numpy as np
import amzn_reviews_cleaner_funcs as amzn
from pyspark.sql import SparkSession

%autoreload 2

<hr>
## Load Data

In [2]:
# create spark session
spark = SparkSession(sc)

# get dataframe
# specify s3 as sourc with s3a://
df = spark.read.json("s3a://amazon-review-data/reviews_Musical_Instruments_5.json.gz")
df.show(3)

+----------+--------+-------+--------------------+-----------+--------------+--------------------+--------------------+--------------+
|      asin| helpful|overall|          reviewText| reviewTime|    reviewerID|        reviewerName|             summary|unixReviewTime|
+----------+--------+-------+--------------------+-----------+--------------+--------------------+--------------------+--------------+
|1384719342|  [0, 0]|    5.0|Not much to write...|02 28, 2014|A2IBPI20UZIR0U|cassandra tu "Yea...|                good|    1393545600|
|1384719342|[13, 14]|    5.0|The product does ...|03 16, 2013|A14VAT5EAX3D9S|                Jake|                Jake|    1363392000|
|1384719342|  [1, 1]|    5.0|The primary job o...|08 28, 2013|A195EZSQDW3E21|Rick Bennette "Ri...|It Does The Job Well|    1377648000|
+----------+--------+-------+--------------------+-----------+--------------+--------------------+--------------------+--------------+
only showing top 3 rows



<hr>
## Test helper module

### Add tfidf vectors

In [3]:
df_tfidf, vocab = amzn.add_tfidf(df)

df_tfidf.select("idf_vector").show(3)

+--------------------+
|          idf_vector|
+--------------------+
|(51989,[3,4,14,18...|
|(51989,[2,3,14,20...|
|(51989,[10,13,24,...|
+--------------------+
only showing top 3 rows



### Extract top n features

In [42]:
n = 10

# get test row
test_row = df_tfidf.first()

# get tfidf vector
test_idf = test_row["idf_vector"]

# get indices for top 10 features
test_term_indices = test_idf.indices[-n:]

In [43]:
# get top n features
test_n_features = [vocab[i] for i in test_term_indices]

test_n_features

[u'sounds.',
 u'recordings',
 u'despite',
 u'here,',
 u'write',
 u'prices',
 u'lowest',
 u'filters',
 u'crisp.',
 u'pricing,']

In [7]:
def extract_top_n_features(tfidf_vector, n=10):
    term_indices = tfidf_vector.indices[-n:]
    features = [vocab[i] for i in term_indices]
    
    return features

In [11]:
from pyspark.sql.functions import udf

# Create udf function to extract n features
extract_n = udf(lambda x: extract_top_n_features(x, 20))

# Apply udf function, get new df with features
test_df_features = df_tfidf.withColumn("top_features", extract_n(df_tfidf["idf_vector"]))

In [9]:
test_df_features.select("top_features").show(3)

+--------------------+
|        top_features|
+--------------------+
|[buy, work, it,, ...|
|[hint, pleasing, ...|
|[mike, otherwise,...|
+--------------------+
only showing top 3 rows



### Test extract top n features

In [6]:
df_features = amzn.add_top_features(df_tfidf, vocab)

In [5]:
df_features.select("top_features").show(3)

Py4JJavaError: An error occurred while calling o115.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 8.0 failed 1 times, most recent failure: Lost task 0.0 in stage 8.0 (TID 7, localhost): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/usr/local/Cellar/apache-spark/2.0.1/libexec/python/lib/pyspark.zip/pyspark/worker.py", line 172, in main
    process()
  File "/usr/local/Cellar/apache-spark/2.0.1/libexec/python/lib/pyspark.zip/pyspark/worker.py", line 167, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/usr/local/Cellar/apache-spark/2.0.1/libexec/python/lib/pyspark.zip/pyspark/worker.py", line 106, in <lambda>
    func = lambda _, it: map(mapper, it)
  File "/usr/local/Cellar/apache-spark/2.0.1/libexec/python/lib/pyspark.zip/pyspark/worker.py", line 92, in <lambda>
    mapper = lambda a: udf(*a)
  File "/usr/local/Cellar/apache-spark/2.0.1/libexec/python/lib/py